# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770363415150                   -0.53    8.0         
  2   -2.771683085936       -2.88       -1.30    1.0    182ms
  3   -2.771714021592       -4.51       -2.64    1.0    178ms
  4   -2.771714712873       -6.16       -3.91    2.0    158ms
  5   -2.771714715053       -8.66       -4.40    2.0    205ms
  6   -2.771714715243       -9.72       -5.62    1.0    142ms
  7   -2.771714715250      -11.22       -6.02    2.0    183ms
  8   -2.771714715250      -12.86       -6.81    2.0    169ms
  9   -2.771714715250   +  -14.75       -7.39    1.0    161ms
 10   -2.771714715250      -13.91       -8.65    2.0    188ms


-0.00013457344528199533

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770498684041                   -0.52    9.0         
  2   -2.771779229967       -2.89       -1.32    1.0    143ms
  3   -2.771801615268       -4.65       -2.44    1.0    223ms
  4   -2.771801997609       -6.42       -3.14    1.0    142ms
  5   -2.771802074279       -7.12       -4.28    2.0    161ms
  6   -2.771802074354      -10.13       -4.40    1.0    182ms
  7   -2.771802074473       -9.92       -5.31    1.0    147ms
  8   -2.771802074475      -11.61       -5.48    1.0    181ms
  9   -2.771802074476      -12.24       -6.27    1.0    156ms
 10   -2.771802074476      -13.14       -6.58    2.0    196ms
 11   -2.771802074476      -14.88       -6.99    1.0    165ms
 12   -2.771802074476   +  -13.85       -8.11    1.0    166ms


0.017612222000930218

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457344528199533
Displaced dipole:  0.017612222000930218
Polarizability :   1.7746795446212211


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920740982e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551476170e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852767579810e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694594424465e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088791643842e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.307639824778e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.543475709608e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.543475709608e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.081379314327e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 6.418131457759e-12
[ Info: GMRES linsolve in iter 2; step 3: normres = 3.420629446666e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 3.420629446666e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 3.42058579194524e-13
└ *  number of operati

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.